In [148]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

gsp = pd.read_excel('Draft-Reference-Network-Mapping-Statement-consultation-2024.xlsx', sheet_name = 'Zones to GSP Groups')
# Map GSP zones to code(1-14)
gsp_mapping = pd.Series(gsp.iloc[1:15,2].values,index=gsp.iloc[1:15,0]).to_dict()

In [149]:
output = pd.DataFrame(columns = ['Datetime (UTC)', 'Eastern', 'East Midlands', 'London',
       'Merseyside & North Wales', 'Midlands', 'Northern', 'North Western',
       'Southern', 'South Eastern', 'South Wales', 'South Western',
       'Yorkshire', 'South of Scotland', 'North of Scotland'])


In [150]:
# Values from all the periods
datasets = ['TLFA-I009_ASZTLF_Spring_A', 'TLFA-I009_ASZTLF_Summer_A', 'TLFA-I009_ASZTLF_Autumn_A 1', 'TLFA-I009_ASZTLF_Winter_A', 'TLFA-I009_ASZTLF_Spring_B']
for d in datasets:
    df = pd.read_csv(d + '.csv')
    # Map the value to the same code to backtrack the GSP name
    value_mapping = pd.Series(df.iloc[0:14,2].values,index=df.iloc[0:14,1]).to_dict()
    # Split start date to convert to datetime
    start = str(df.iloc[1,3])
    start_date = datetime(int(start[0:4]), int(start[4:6]), int(start[6:8]))
    # Split end date to convert to datetime
    end = str(df.iloc[1,4])
    end_date = datetime(int(end[0:4]), int(end[4:6]), int(end[6:8]),23,30)

    dates_range = []
    # Increment every 30 minutes from start date to end date
    while start_date <= end_date:
        dates_range.append(start_date)
        start_date += timedelta(minutes=30)
    
    temp = pd.DataFrame({'Datetime (UTC)': dates_range})
    # Map the values to each GSP name column
    for k in gsp_mapping.keys():
        temp[gsp_mapping[k]] = value_mapping[int(k)]
    # Concatenate results for all periods
    output = pd.concat([output, temp])


In [152]:
output.to_csv('GSP.csv', index = False)